In [1]:
import pandas as pd
import os.path
import numpy as np
import time

### Read Dataset

In [2]:
df = pd.DataFrame() # Initialize an empty dataframe to store the results of training for hyperparmeter visualization
data=pd.read_csv('dataset.csv',header=None)
data.head()

,0,1,2,3,4
0,3.62160,8.6661,-2.8073,-0.44699,0
1,4.54590,8.1674,-2.4586,-1.46210,0
2,3.86600,-2.6383,1.9242,0.10645,0
3,3.45660,9.5228,-4.0112,-3.59440,0
4,0.32924,-4.4552,4.5718,-0.98880,0


In [3]:
y=np.array(data.iloc[:,-1])
x=np.array(data.iloc[:,:-1]).T
print('Target variable: ', y)
print('Shape of Y: ', y.shape)
print('Shape of X: ', x.shape)
# y_new = y.copy()

Target variable:  [0 0 0 ... 1 1 1]
Shape of Y:  (1372,)
Shape of X:  (4, 1372)


### Hyperparameters

In [4]:
# Basic hyperparameters are defined here. The activation functions and number of neurons in each layer is configurable by
# the user and entered during the start of the program.
np.random.seed(420)
no_hidden_layers = 2     # The number of hidden layers required
learning_rate = 0.01     # The learning rate for the ANN
no_epochs = 3000          # Number of iterations to train the ANN
number_of_inputs = 4     # Input neurons, defined by the dataset
output_neurons = 1       # Number of output neurons to use

max_no_hidden_layers = 5 # This is the maximum number of hidden layers to be used for testing, we chose the upper limit as
                         # 5 from the literature survery we performed. Can be increased to test Deep Neural Networks.

### Creating a dictionary with hyperparameters

In [5]:
# This function initializes an empty dictionary that stores each of the hyperparameters and other params such as accuracy, 
# time taken and other metrics for data visualization later and to see which hyperparameters give us the best results.

def init_hyperparams_dict(max_no_hidden_layers):
    keys = ['no_hidden_layers']    # List that contains the keys for the dictionary
    for i in range(max_no_hidden_layers):    # Create keys for each hidden layer
        keys.append('no_neurons_hidden_layer_'+str(i))
        keys.append('activation_function_'+str(i))

    keys.extend(['output_activation_function', 'learning_rate', 'no_epochs', 'accuracy', 'time'])
    hyperparams = dict.fromkeys(keys)    # Create a dictionary with the given keys
    return hyperparams

# This function adds in values to the dictionary and takes input from the user for each of the activation function to be 
# used and the number of neurons in each hidden layer. It also adds other hyperparameters to the dictionary for storage.

def add_params_dict(hyperparams, no_hidden_layers, max_no_hidden_layers, learning_rate, no_epochs):
    hyperparams['no_hidden_layers'] = no_hidden_layers
    i = 0
    
    # Input number of neurons for each layer and their acitvation functions from the user.
    while i < max_no_hidden_layers:
        if i < no_hidden_layers:    # Only take input for the given number of hidden layers 
            hyperparams['no_neurons_hidden_layer_'+str(i)] = int(input('Enter number of neurons in Layer'+str(i+1)+': '))
            hyperparams['activation_function_'+str(i)] = str(input('Enter the Activation Function for Layer'+str(i+1)+': '))
        else:                     # The values for other unused layers will be initialized to none.
            hyperparams["no_neurons_hidden_layer_"+str(i)] = None
            hyperparams["activation_function_"+str(i)] = None
        i += 1
    hyperparams['output_activation_function'] = str(input('Enter the Activation Function for output: '))
    hyperparams['learning_rate'] = learning_rate
    hyperparams['no_epochs'] = no_epochs
    return hyperparams

# After the code has run, we obtain the accuracy and time required to run it. Once the code has run, this function is called
# that stores the accuracy and time taken for this specific configuration of NN in the dictionary.
def add_params_dict_1(hyperparams, accuracy, time):
    hyperparams['accuracy'] = accuracy
    hyperparams['time'] = time
    return hyperparams

# This function takes all the parameters of the NN and writes it to a CSV file for later analysis.
def write_params(hyperparams, df):
    df = df.append(hyperparams, ignore_index=True)
    if(os.path.isfile('hyperparams.csv')):
        df_old = pd.read_csv('hyperparams.csv')
        df_old = df_old.append(df)
        df_old = df_old.drop_duplicates(keep='first', inplace=False, ignore_index=True)
        df_old.to_csv('hyperparams.csv', index = False)
    else:
        df.to_csv('hyperparams.csv', index = False)

In [6]:
# Initializing our hyperparameter dictionary that stores each aspect of the NN.
hyperparams = init_hyperparams_dict(max_no_hidden_layers)
hyperparams = add_params_dict(hyperparams, no_hidden_layers, max_no_hidden_layers, learning_rate, no_epochs)

Enter number of neurons in Layer1: 5
Enter the Activation Function for Layer1: sigmoid
Enter number of neurons in Layer2: 10
Enter the Activation Function for Layer2: sigmoid
Enter the Activation Function for output: relu


In [7]:
hyperparams

{'no_hidden_layers': 2,
 'no_neurons_hidden_layer_0': 5,
 'activation_function_0': 'sigmoid',
 'no_neurons_hidden_layer_1': 10,
 'activation_function_1': 'sigmoid',
 'no_neurons_hidden_layer_2': None,
 'activation_function_2': None,
 'no_neurons_hidden_layer_3': None,
 'activation_function_3': None,
 'no_neurons_hidden_layer_4': None,
 'activation_function_4': None,
 'output_activation_function': 'relu',
 'learning_rate': 0.01,
 'no_epochs': 3000,
 'accuracy': None,
 'time': None}

In [8]:
# Initialize weights and biases array to store the weights and biases.
weights = []
bias = []

# This function generates the weights and biases for the NN based on the number of neurons and layers defined by the user.
def generate_wb(number_of_inputs, hyperparams):
    # Generate W1
    weights.append(np.random.uniform(low=-0.5, high=0.5, size=(number_of_inputs, hyperparams['no_neurons_hidden_layer_0'])))
    i = 0
    while i<hyperparams['no_hidden_layers']:
        if(i == hyperparams['no_hidden_layers'] - 1): 
            # For the weights and biases of last layer use number of output neurons.
            weights.append(np.random.uniform(low=-0.5,high=0.5,size=(hyperparams['no_neurons_hidden_layer_'+str(i)], output_neurons)))
            bias.append(np.zeros((hyperparams['no_neurons_hidden_layer_'+str(i)], 1)))
        else:
            # For middle hidden layers, use the current number of neurons and next layer number of neurons to create Weights and Biases
            weights.append(np.random.uniform(low=-0.5,high=0.5,size=(hyperparams['no_neurons_hidden_layer_'+str(i)], hyperparams['no_neurons_hidden_layer_'+str(i+1)])))
            bias.append(np.zeros((hyperparams['no_neurons_hidden_layer_'+str(i)], 1)))
        i+=1
    bias.append(np.zeros((output_neurons, 1)))
    return weights, bias

### Acitvation Functions

In [9]:
def sigmoid(z):
    a=np.zeros([1,1])
    a=1/(1+np.exp(-z))
    return a

def sigmoid_derivative(A):
    return A*(1-A)
    
def ReLU(z):
    return np.maximum(z, 0)

def ReLU_derivative(z):
    return z > 0

def tanh(z):
    z=(np.exp(z)-np.exp(-z))/(np.exp(z)+np.exp(-z))
    z=1-z**2
    return z

def tanh_derivative(z):
    dz=1-z**2
    return dz

### Forward Propagation

In [10]:
# This function does forward propagation using the user inputted values from the hyperparams dictionary.
def forward_propagation(weights, bias, hyperparams, x):
    z = []    # Initialize z & a matrix that store the respective Z1,Z2.....Zn and A1, A2.....An values
    a = []
    i = 0
    while i <= hyperparams['no_hidden_layers']:
        if i == 0:
            z.append(np.matmul(weights[i].T,x) + bias[i])   # For the first layer, use the input x
        else:
            z.append(np.matmul(weights[i].T, a[i - 1]) + bias[i]) # For the layers after 1st, use output from previous layer
        if i == hyperparams['no_hidden_layers']:   # At last output layer check for the output activation function and use that
            if(hyperparams['output_activation_function'] == 'sigmoid'):
                a.append(sigmoid(z[i]))
            elif(hyperparams['output_activation_function'] == 'tanh'):
                a.append(tanh(z[i]))
            elif(hyperparams['output_activation_function'] == 'relu'):
                a.append(ReLU(z[i]))
        else:      # For other layers, use the activation function of that layer from the hyperparams dictionary
            if(hyperparams['activation_function_'+str(i)] == 'sigmoid'):
                a.append(sigmoid(z[i]))
            elif(hyperparams['activation_function_'+str(i)] == 'tanh'):
                a.append(tanh(z[i]))
            elif(hyperparams['activation_function_'+str(i)] == 'relu'):
                a.append(ReLU(z[i]))
        
        i+=1
    
    return z, a

### Log loss

In [11]:
# This function simply calculates the log loss and returns it
def log_loss_calc(output, y):
    loss = 0
    loss =- 1 * np.sum(y * np.log(output + 1e-10) + ((1 - y) * np.log(1 - output + 1e-10)))
    loss = loss / y.shape[0]
    return loss

### Hinge Loss

In [12]:
# This calculates the Hinge Loss from the NN and returns it. Hinge loss works best with target variables in [-1,1] range.
# Thus we change our output variable y from [0,1] to [-1,1]
def hinge_loss_calc(y, y_hat):
    y_new = np.array([-1 if i==0 else i for i in y])        # Replace [0,1] in y to [-1,1]
    y_hat = np.array([-1 if i==0 else i for i in y_hat])    # Replace [0,1] in y_hat to [-1,1]
    hinge_loss = np.mean([max(0, 1-x*y) for x, y in zip(y_new, y_hat)])
    return hinge_loss


### Backward Propagation

In [13]:
# Initialize dw and db arrays that will store the biases and weights for each hidden layer of the NN as configured by the user.
dw = []
db = []
def generate_dwdb(hyperparams):
    dw.clear()
    db.clear()
    for i in range(hyperparams['no_hidden_layers']):
        dw.append(np.zeros_like(weights[i]))
        db.append(np.zeros_like(bias[i]))
    return dw, db

In [14]:
# Back propagation function
# Since this code is dynamic and the number of hidden layers and neurons in each layer depend on the value input by user
# thus, the dz, dw, db are stored in a list. In backward propagation we calculate dz(n), dw(n), db(n) first, the indexing
# of these arrays is opposite now.
# For example with a NN of 3 Hidden Layers and 1 Output, dz[3] is stored in dz[0], dz[2] in dz[1], dz[1] in dz[2] & finally
# dz[0] in dz[3]. The same applies for dw and db lists. Thus after all the calculation at the end we have reversed the lists
# to get correct ordering of dz,dw,db for correct new weights and biases calculation.

def backward_propagation(a, weights, bias, y, x):
    dz = []
    dz.append((a[no_hidden_layers]) - y)   # This is stored in dz[0] but is actually dz[no_hidden_layers]
    dw.append(np.matmul(a[no_hidden_layers - 1],dz[0].T)/y.shape[0])
    db.append(np.sum(dz[0],axis=0,keepdims=True)/y.shape[0])

    i = no_hidden_layers    # This variable is used to access the weights and the activation functions of that layer.
    j = 1    # This variable is used to keep track of and access dz.
    while i>0:
    
        if(hyperparams['activation_function_'+str(i-1)] == 'sigmoid'):
            dz.append(np.multiply(np.matmul(weights[i],dz[j-1]),sigmoid_derivative(a[i-1])))
        elif(hyperparams['activation_function_'+str(i-1)] == 'tanh'):
            dz.append(np.multiply(np.matmul(weights[i],dz[j-1]),tanh_derivative(a[i-1])))
        elif(hyperparams['activation_function_'+str(i-1)] == 'relu'):
            dz.append(np.multiply(np.matmul(weights[i],dz[j-1]),ReLU_derivative(a[i-1])))
        if(i!=1):
            dw.append(np.matmul(a[i-2], dz[j].T)/y.shape[0])
            db.append(np.sum(dz[j],axis=0, keepdims=True)/y.shape[0])
        elif(i==1):  # When we reach the first layer at the end of back propagation, we use the input and not output from
                     # pervious layer, thus this has its own if statement.
            dw.append(np.matmul(x,dz[j].T)/y.shape[0])
            db.append(np.sum(dz[j], axis=0, keepdims=True)/y.shape[0])
        i-=1
        j+=1
    dz.reverse()
    db.reverse()
    dw.reverse()
    
    i=0
    # The while loop simply updates the value of weight and bias at each index.
    while i<=no_hidden_layers:
        weights[i] = weights[i]-learning_rate*dw[i]
        bias[i] = bias[i]-learning_rate*db[i]
        i+=1
        
    return weights, bias, dw, db

In [15]:
# This function calculates the accuracy after each epoch.
def accuracy_calc(output):
    y_hat=[]
    correct=0
    accuracy=0
    for i, m in enumerate(output):
        if m<0.5:
            y_hat.append(0)
        elif m>=0.5:
            y_hat.append(1)
            
    correct=np.sum(y==y_hat)
    accuracy=correct/y.shape[0]
    return y_hat, correct, accuracy

In [16]:
t=time.time()    # This measures the time taken by the NN to converge at the given number of epochs.
weights, bias = generate_wb(number_of_inputs, hyperparams)    # Initiliaze the weights and biases array.
for j in range(no_epochs):                                    # Run the NN for the given number of epochs.
    dw, db = generate_dwdb(hyperparams)                       # Generate dw and db lists for each layer.
    z, a = forward_propagation(weights, bias, hyperparams, x) # Perform Forward Propagation
    log_loss = log_loss_calc(a[no_hidden_layers][0], y)            # Calculate log loss of the NN.
    weights, bias, dw, db = backward_propagation(a, weights, bias, y, x)   # Perform Backward Propagation
    y_hat, correct, accuracy = accuracy_calc(a[no_hidden_layers][0])
#     y_hat=[]
#     correct=0
#     accuracy=0
#     for i, m in enumerate(a[no_hidden_layers][0]):
#         if m<0.5:
#             y_hat.append(-1)
#         elif m>=0.5:
#             y_hat.append(1)
            
    hinge_loss = hinge_loss_calc(y,y_hat)
    print('Total Correct Predictions: ',correct, '/', y.shape[0])
    print('Accuracy for the current Epoch {} = {:.2f}%'.format(j,accuracy*100))

print('*'*50, 'Training Complete', '*'*50)
print('time taken: ' ,time.time()-t)
print('Final LOG cost after training: {:.2f}'.format(log_loss))
print('Final HINGE loss after training : {:.2f}'.format(hinge_loss))
print('Final accuracy after training for {} epochs = {:.2f}%'.format((j + 1),(accuracy * 100)))

Total Correct Predictions:  762 / 1372
Accuracy for the current Epoch 0 = 55.54%
Total Correct Predictions:  762 / 1372
Accuracy for the current Epoch 1 = 55.54%
Total Correct Predictions:  762 / 1372
Accuracy for the current Epoch 2 = 55.54%
Total Correct Predictions:  762 / 1372
Accuracy for the current Epoch 3 = 55.54%
Total Correct Predictions:  762 / 1372
Accuracy for the current Epoch 4 = 55.54%
Total Correct Predictions:  762 / 1372
Accuracy for the current Epoch 5 = 55.54%
Total Correct Predictions:  762 / 1372
Accuracy for the current Epoch 6 = 55.54%
Total Correct Predictions:  762 / 1372
Accuracy for the current Epoch 7 = 55.54%
Total Correct Predictions:  762 / 1372
Accuracy for the current Epoch 8 = 55.54%
Total Correct Predictions:  762 / 1372
Accuracy for the current Epoch 9 = 55.54%
Total Correct Predictions:  762 / 1372
Accuracy for the current Epoch 10 = 55.54%
Total Correct Predictions:  762 / 1372
Accuracy for the current Epoch 11 = 55.54%
Total Correct Predictions:

Total Correct Predictions:  762 / 1372
Accuracy for the current Epoch 111 = 55.54%
Total Correct Predictions:  762 / 1372
Accuracy for the current Epoch 112 = 55.54%
Total Correct Predictions:  762 / 1372
Accuracy for the current Epoch 113 = 55.54%
Total Correct Predictions:  762 / 1372
Accuracy for the current Epoch 114 = 55.54%
Total Correct Predictions:  762 / 1372
Accuracy for the current Epoch 115 = 55.54%
Total Correct Predictions:  762 / 1372
Accuracy for the current Epoch 116 = 55.54%
Total Correct Predictions:  762 / 1372
Accuracy for the current Epoch 117 = 55.54%
Total Correct Predictions:  762 / 1372
Accuracy for the current Epoch 118 = 55.54%
Total Correct Predictions:  762 / 1372
Accuracy for the current Epoch 119 = 55.54%
Total Correct Predictions:  762 / 1372
Accuracy for the current Epoch 120 = 55.54%
Total Correct Predictions:  762 / 1372
Accuracy for the current Epoch 121 = 55.54%
Total Correct Predictions:  762 / 1372
Accuracy for the current Epoch 122 = 55.54%
Tota

Total Correct Predictions:  921 / 1372
Accuracy for the current Epoch 217 = 67.13%
Total Correct Predictions:  923 / 1372
Accuracy for the current Epoch 218 = 67.27%
Total Correct Predictions:  922 / 1372
Accuracy for the current Epoch 219 = 67.20%
Total Correct Predictions:  923 / 1372
Accuracy for the current Epoch 220 = 67.27%
Total Correct Predictions:  923 / 1372
Accuracy for the current Epoch 221 = 67.27%
Total Correct Predictions:  924 / 1372
Accuracy for the current Epoch 222 = 67.35%
Total Correct Predictions:  924 / 1372
Accuracy for the current Epoch 223 = 67.35%
Total Correct Predictions:  924 / 1372
Accuracy for the current Epoch 224 = 67.35%
Total Correct Predictions:  924 / 1372
Accuracy for the current Epoch 225 = 67.35%
Total Correct Predictions:  925 / 1372
Accuracy for the current Epoch 226 = 67.42%
Total Correct Predictions:  929 / 1372
Accuracy for the current Epoch 227 = 67.71%
Total Correct Predictions:  930 / 1372
Accuracy for the current Epoch 228 = 67.78%
Tota

Total Correct Predictions:  928 / 1372
Accuracy for the current Epoch 323 = 67.64%
Total Correct Predictions:  928 / 1372
Accuracy for the current Epoch 324 = 67.64%
Total Correct Predictions:  928 / 1372
Accuracy for the current Epoch 325 = 67.64%
Total Correct Predictions:  928 / 1372
Accuracy for the current Epoch 326 = 67.64%
Total Correct Predictions:  928 / 1372
Accuracy for the current Epoch 327 = 67.64%
Total Correct Predictions:  928 / 1372
Accuracy for the current Epoch 328 = 67.64%
Total Correct Predictions:  928 / 1372
Accuracy for the current Epoch 329 = 67.64%
Total Correct Predictions:  928 / 1372
Accuracy for the current Epoch 330 = 67.64%
Total Correct Predictions:  928 / 1372
Accuracy for the current Epoch 331 = 67.64%
Total Correct Predictions:  928 / 1372
Accuracy for the current Epoch 332 = 67.64%
Total Correct Predictions:  928 / 1372
Accuracy for the current Epoch 333 = 67.64%
Total Correct Predictions:  928 / 1372
Accuracy for the current Epoch 334 = 67.64%
Tota

Total Correct Predictions:  934 / 1372
Accuracy for the current Epoch 472 = 68.08%
Total Correct Predictions:  934 / 1372
Accuracy for the current Epoch 473 = 68.08%
Total Correct Predictions:  934 / 1372
Accuracy for the current Epoch 474 = 68.08%
Total Correct Predictions:  934 / 1372
Accuracy for the current Epoch 475 = 68.08%
Total Correct Predictions:  934 / 1372
Accuracy for the current Epoch 476 = 68.08%
Total Correct Predictions:  934 / 1372
Accuracy for the current Epoch 477 = 68.08%
Total Correct Predictions:  935 / 1372
Accuracy for the current Epoch 478 = 68.15%
Total Correct Predictions:  935 / 1372
Accuracy for the current Epoch 479 = 68.15%
Total Correct Predictions:  935 / 1372
Accuracy for the current Epoch 480 = 68.15%
Total Correct Predictions:  935 / 1372
Accuracy for the current Epoch 481 = 68.15%
Total Correct Predictions:  935 / 1372
Accuracy for the current Epoch 482 = 68.15%
Total Correct Predictions:  935 / 1372
Accuracy for the current Epoch 483 = 68.15%
Tota

...

Total Correct Predictions:  1314 / 1372
Accuracy for the current Epoch 2555 = 95.77%
Total Correct Predictions:  1314 / 1372
Accuracy for the current Epoch 2556 = 95.77%
Total Correct Predictions:  1314 / 1372
Accuracy for the current Epoch 2557 = 95.77%
Total Correct Predictions:  1314 / 1372
Accuracy for the current Epoch 2558 = 95.77%
Total Correct Predictions:  1314 / 1372
Accuracy for the current Epoch 2559 = 95.77%
Total Correct Predictions:  1314 / 1372
Accuracy for the current Epoch 2560 = 95.77%
Total Correct Predictions:  1314 / 1372
Accuracy for the current Epoch 2561 = 95.77%
Total Correct Predictions:  1314 / 1372
Accuracy for the current Epoch 2562 = 95.77%
Total Correct Predictions:  1314 / 1372
Accuracy for the current Epoch 2563 = 95.77%
Total Correct Predictions:  1314 / 1372
Accuracy for the current Epoch 2564 = 95.77%
Total Correct Predictions:  1314 / 1372
Accuracy for the current Epoch 2565 = 95.77%
Total Correct Predictions:  1314 / 1372
Accuracy for the current 

Total Correct Predictions:  1315 / 1372
Accuracy for the current Epoch 2655 = 95.85%
Total Correct Predictions:  1315 / 1372
Accuracy for the current Epoch 2656 = 95.85%
Total Correct Predictions:  1315 / 1372
Accuracy for the current Epoch 2657 = 95.85%
Total Correct Predictions:  1315 / 1372
Accuracy for the current Epoch 2658 = 95.85%
Total Correct Predictions:  1315 / 1372
Accuracy for the current Epoch 2659 = 95.85%
Total Correct Predictions:  1315 / 1372
Accuracy for the current Epoch 2660 = 95.85%
Total Correct Predictions:  1315 / 1372
Accuracy for the current Epoch 2661 = 95.85%
Total Correct Predictions:  1315 / 1372
Accuracy for the current Epoch 2662 = 95.85%
Total Correct Predictions:  1315 / 1372
Accuracy for the current Epoch 2663 = 95.85%
Total Correct Predictions:  1315 / 1372
Accuracy for the current Epoch 2664 = 95.85%
Total Correct Predictions:  1315 / 1372
Accuracy for the current Epoch 2665 = 95.85%
Total Correct Predictions:  1315 / 1372
Accuracy for the current 

Total Correct Predictions:  1316 / 1372
Accuracy for the current Epoch 2758 = 95.92%
Total Correct Predictions:  1316 / 1372
Accuracy for the current Epoch 2759 = 95.92%
Total Correct Predictions:  1316 / 1372
Accuracy for the current Epoch 2760 = 95.92%
Total Correct Predictions:  1316 / 1372
Accuracy for the current Epoch 2761 = 95.92%
Total Correct Predictions:  1316 / 1372
Accuracy for the current Epoch 2762 = 95.92%
Total Correct Predictions:  1316 / 1372
Accuracy for the current Epoch 2763 = 95.92%
Total Correct Predictions:  1316 / 1372
Accuracy for the current Epoch 2764 = 95.92%
Total Correct Predictions:  1316 / 1372
Accuracy for the current Epoch 2765 = 95.92%
Total Correct Predictions:  1316 / 1372
Accuracy for the current Epoch 2766 = 95.92%
Total Correct Predictions:  1316 / 1372
Accuracy for the current Epoch 2767 = 95.92%
Total Correct Predictions:  1316 / 1372
Accuracy for the current Epoch 2768 = 95.92%
Total Correct Predictions:  1316 / 1372
Accuracy for the current 

Total Correct Predictions:  1319 / 1372
Accuracy for the current Epoch 2861 = 96.14%
Total Correct Predictions:  1319 / 1372
Accuracy for the current Epoch 2862 = 96.14%
Total Correct Predictions:  1319 / 1372
Accuracy for the current Epoch 2863 = 96.14%
Total Correct Predictions:  1319 / 1372
Accuracy for the current Epoch 2864 = 96.14%
Total Correct Predictions:  1319 / 1372
Accuracy for the current Epoch 2865 = 96.14%
Total Correct Predictions:  1319 / 1372
Accuracy for the current Epoch 2866 = 96.14%
Total Correct Predictions:  1319 / 1372
Accuracy for the current Epoch 2867 = 96.14%
Total Correct Predictions:  1320 / 1372
Accuracy for the current Epoch 2868 = 96.21%
Total Correct Predictions:  1320 / 1372
Accuracy for the current Epoch 2869 = 96.21%
Total Correct Predictions:  1320 / 1372
Accuracy for the current Epoch 2870 = 96.21%
Total Correct Predictions:  1320 / 1372
Accuracy for the current Epoch 2871 = 96.21%
Total Correct Predictions:  1320 / 1372
Accuracy for the current 

Total Correct Predictions:  1321 / 1372
Accuracy for the current Epoch 2962 = 96.28%
Total Correct Predictions:  1321 / 1372
Accuracy for the current Epoch 2963 = 96.28%
Total Correct Predictions:  1321 / 1372
Accuracy for the current Epoch 2964 = 96.28%
Total Correct Predictions:  1321 / 1372
Accuracy for the current Epoch 2965 = 96.28%
Total Correct Predictions:  1321 / 1372
Accuracy for the current Epoch 2966 = 96.28%
Total Correct Predictions:  1321 / 1372
Accuracy for the current Epoch 2967 = 96.28%
Total Correct Predictions:  1321 / 1372
Accuracy for the current Epoch 2968 = 96.28%
Total Correct Predictions:  1321 / 1372
Accuracy for the current Epoch 2969 = 96.28%
Total Correct Predictions:  1321 / 1372
Accuracy for the current Epoch 2970 = 96.28%
Total Correct Predictions:  1321 / 1372
Accuracy for the current Epoch 2971 = 96.28%
Total Correct Predictions:  1321 / 1372
Accuracy for the current Epoch 2972 = 96.28%
Total Correct Predictions:  1321 / 1372
Accuracy for the current 

In [17]:
# Store the time taken by the program in variable time and add it to the hyperparams dictionary along with accuracy of the NN.
time = time.time()-t
add_params_dict_1(hyperparams, accuracy, time)
# Please ensure that the file 'hyperparams.csv' is closed before running this script or it raises a "Permission Denied" error.
write_params(hyperparams, df)